Установка необходимых библиотек:

In [1]:
import pandas
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import spacy
!pip install natasha
from natasha import NewsSyntaxParser
from natasha import NewsEmbedding
from natasha import Segmenter
from natasha import Doc
!pip install razdel
from razdel import sentenize
emb = NewsEmbedding()
syntax_parser = NewsSyntaxParser(emb)
segmenter = Segmenter()
from bs4 import BeautifulSoup
!pip install wiki-ru-wordnet
from wiki_ru_wordnet import WikiWordnet
wikiwordnet = WikiWordnet()
import numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=094706044559ca0e0ec9b56d2ec22f98ad26ed32689a6041863a87f876281b2e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26099 sha256=9d049b430024e6212520c1cd5e250ea084ad0c0a7f2259d461e91e506759fa3f
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 50.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wiki-ru-wordnet: filename=wiki_ru_wordnet-1.0.3-py3-no

Составление таблицы-словаря с зависимыми словами:

In [2]:
valency_examples_table = pandas.read_excel('/content/Словарь валентностей.xlsx', sheet_name='Примеры')

In [3]:
valency_examples_table

,Несовершенный вид,Совершенный вид,Значение,Предлог,Часть речи,Падеж,Примеры
0,возводить,возвести,Математическое,в,NOUN,accs,"степень, квадрат, куб"
1,возводить,возвести,Нематематическое,NaN,NOUN,accs,"дворец, трамплин, горку, арену, мансарду, стад..."
2,возводить,возвести,Нематематическое,в,NOUN,accs,"атрибут, принцип, культ, закономерность, систе..."
3,возводить,возвести,Нематематическое,к,NOUN,datv,"Кеплеру, тексту"
4,возводить,возвести,Нематематическое,на,NOUN,accs,"небеса, пьедестал, уровень, помост, трон, престол"
...,...,...,...,...,...,...,...
208,экспонировать,экспонировать,Математическое,NaN,NOUN,accs,"число, переменную, функцию"
209,экспонировать,экспонировать,Математическое,NaN,NUMR,accs,"число, переменную, функцию"
210,экспонировать,экспонировать,Нематематическое,NaN,NOUN,accs,"картину, портрет, произведение, пейзаж, живопи..."
211,экспонировать,экспонировать,Нематематическое,в,NOUN,loct,"зале, музее, Эрмитаже, Третьяковке, России, эк..."


В словаре примеры зависимых слов стоят не в начальной форме. Нужно поставить каждое слово в начальную форму:

In [4]:
num_rows = valency_examples_table.count()['Примеры']
for i in range(0, num_rows):
    examples = valency_examples_table['Примеры'].values[i]
    examples = examples.split(', ')
    length = len(examples)
    for k in range(0, length):
        example = examples[k]
        lemma = morph.parse(example)[0]
        lemma = lemma.normal_form
        examples[k] = lemma
    valency_examples_table.loc[i, 'Примеры'] = examples

Таблица обновилась. Теперь все примеры стоят в начальной форме:

In [5]:
valency_examples_table

,Несовершенный вид,Совершенный вид,Значение,Предлог,Часть речи,Падеж,Примеры
0,возводить,возвести,Математическое,в,NOUN,accs,"[степень, квадрат, куб]"
1,возводить,возвести,Нематематическое,NaN,NOUN,accs,"[дворец, трамплин, горка, арена, мансарда, ста..."
2,возводить,возвести,Нематематическое,в,NOUN,accs,"[атрибут, принцип, культ, закономерность, сист..."
3,возводить,возвести,Нематематическое,к,NOUN,datv,"[кеплер, текст]"
4,возводить,возвести,Нематематическое,на,NOUN,accs,"[небо, пьедестал, уровень, помост, трон, престол]"
...,...,...,...,...,...,...,...
208,экспонировать,экспонировать,Математическое,NaN,NOUN,accs,"[число, переменный, функция]"
209,экспонировать,экспонировать,Математическое,NaN,NUMR,accs,"[число, переменный, функция]"
210,экспонировать,экспонировать,Нематематическое,NaN,NOUN,accs,"[картина, портрет, произведение, пейзаж, живоп..."
211,экспонировать,экспонировать,Нематематическое,в,NOUN,loct,"[зал, музей, эрмитаж, третьяковка, россия, экс..."


Составление таблицы-словаря с семантическими классами:

In [7]:
valency_semantics_table = pandas.read_excel('/content/Словарь валентностей.xlsx', sheet_name='Семантика')

In [8]:
valency_semantics_table

,Несовершенный вид,Совершенный вид,Значение,Семантика
0,возводить,возвести,Математическое,Степень
1,возводить,возвести,Нематематическое,Строение
2,возводить,возвести,Нематематическое,Часть тела
3,возводить,возвести,Нематематическое,Обвинение
4,возводить,возвести,Нематематическое,Закон
...,...,...,...,...
351,экспонировать,экспонировать,Математическое,Число
352,экспонировать,экспонировать,Нематематическое,Искусство
353,экспонировать,экспонировать,Нематематическое,Объект
354,экспонировать,экспонировать,Нематематическое,Объект (место)


Семантический класс "Объект" мало о чём говорит для анализатора. Его нужно удалить:

In [9]:
num_rows = valency_semantics_table.count()['Семантика']
indexes_to_delete = []
for i in range(0, num_rows):
    semantics = valency_semantics_table['Семантика'].values[i]
    index = semantics.find('Объект')
    if index > -1:
        indexes_to_delete.append(i)
valency_semantics_table = valency_semantics_table.drop(index = indexes_to_delete)

Таблицу после этого нужно переиндексировать:

In [10]:
new_index = 0
for index, row in valency_semantics_table.iterrows():
    valency_semantics_table.rename(index = {index: new_index}, inplace = True)
    new_index += 1

К имеющимся семантическим классам добавим синонимичные, чтобы охватить как можно больше значений:

In [11]:
for index, row in valency_semantics_table.iterrows():
    semantics = valency_semantics_table['Семантика'].values[index]
    semantics = semantics.lower()
    semantics = semantics.split()
    valency_semantics_table['Семантика'].values[index] = semantics
    synsets = wikiwordnet.get_synsets(semantics[0])
    for synset in synsets:
        for word in synset.get_words():
            lemma = word.lemma()
            valency_semantics_table['Семантика'].values[index].append(lemma)
    valency_semantics_table['Семантика'].values[index] = list(set(valency_semantics_table['Семантика'].values[index]))

Таблица обновилась. Теперь в ней больше похожих семантических классов:

In [12]:
valency_semantics_table

,Несовершенный вид,Совершенный вид,Значение,Семантика
0,возводить,возвести,Математическое,"[степень, показатель]"
1,возводить,возвести,Нематематическое,"[строение, дом, корпус, постройка, здание]"
2,возводить,возвести,Нематематическое,"[часть, доля, тела, порция]"
3,возводить,возвести,Нематематическое,[обвинение]
4,возводить,возвести,Нематематическое,"[закон, закономерность]"
...,...,...,...,...
223,убывать,убыть,Нематематическое,"[место, площадь, местоположение, положение]"
224,экспонировать,экспонировать,Математическое,"[число, количество]"
225,экспонировать,экспонировать,Математическое,"[число, количество]"
226,экспонировать,экспонировать,Нематематическое,[искусство]


Составление таблицы-словаря с переводом на английский:

In [13]:
english_table = pandas.read_excel('/content/Словарь валентностей.xlsx', sheet_name='Перевод_английский')

In [14]:
english_table

,Несовершенный вид,Совершенный вид,Значение,Перевод
0,возводить,возвести,Математическое,raise
1,вписывать,вписать,Математическое,inscribe
2,выделять,выделить,Математическое1,express
3,выделять,выделить,Математическое2,complete
4,выкалывать,выколоть,Математическое,exclude
5,выносить,вынести,Математическое,factor out
6,вычитать,вычесть,Математическое,subtract
7,дифференцировать,дифференцировать,Математическое,differentiate
8,задавать,задать,Математическое,specify
9,заключать,заключить,Математическое,enclose


Составление таблицы-словаря с переводом на французский:

In [15]:
french_table = pandas.read_excel('/content/Словарь валентностей.xlsx', sheet_name='Перевод_французский')

In [16]:
french_table

,Несовершенный вид,Совершенный вид,Значение,Перевод
0,возводить,возвести,Математическое,élever
1,вписывать,вписать,Математическое,inscrire
2,выделять,выделить,Математическое1,déterminer
3,выделять,выделить,Математическое2,complèter
4,выкалывать,выколоть,Математическое,exclure
5,выносить,вынести,Математическое,mettre en facteur
6,вычитать,вычесть,Математическое,soustraire
7,дифференцировать,дифференцировать,Математическое,différencier
8,задавать,задать,Математическое,donner
9,заключать,заключить,Математическое,placer


Объявление функций:

In [17]:
#Поиск глаголов
def find_verbs(processed_document):
    verbs = []
    for word in processed_document:
        verb = []
        text = word.text
        lemma = morph.parse(text)[0]
        POS = lemma.tag.POS
        if (POS == 'VERB') or (POS == 'INFN'):
            verb.append(lemma.normal_form)
            id = word.id
            verb.append(id)
            verbs.append(verb)
    return verbs

In [18]:
#Поиск зависимых слов
def find_dependents(verbs_list, tokens):
    for verb in verbs_list:
        dependents = []
        id = verb[1]
        for token in tokens:
            head_id = token.head_id
            if head_id == id:
                text_token = token.text
                lemma = morph.parse(text_token)[0]
                POS = lemma.tag.POS
                if POS == ('NOUN' or 'NUMR' or 'NPRO'):
                    id_token = [token.id]
                    case_token = lemma.tag.case
                    id_token.append(POS)
                    id_token.append(case_token)
                    dependents.append(id_token)
        verb.append(dependents)
    return verbs_list

In [19]:
#Поиск предлогов
def find_prepositions(verbs_list, tokens):
    for verb in verbs_list:
        dependents = verb[2]
        for dependent in dependents:
            dependent_id = dependent[0]
            prepositions = []
            for token in tokens:
                head_id = token.head_id
                if head_id == dependent_id:
                    text_token = token.text
                    lemma = morph.parse(text_token)[0]
                    POS = lemma.tag.POS
                    if POS == 'PREP':
                        id_token = token.id
                        prepositions.append(id_token)
            dependent.append(prepositions)
    return verbs_list

In [20]:
#Расшифровка зависимых слов
def decipher_dependents(verbs_list, tokens):
    for verb in verbs_list:
        dependents = verb[2]
        for dependent in dependents:
            dependent_id = dependent[0]
            for token in tokens:
                if token.id == dependent_id:
                    text_token = token.text
                    lemma = morph.parse(text_token)[0]
                    dependent[0] = lemma.normal_form
    return verbs_list 

In [21]:
#Расшифровка предлогов
def decipher_prepositions(verbs_list, tokens):
    for verb in verbs_list:
        dependents = verb[2]
        for dependent in dependents:
            prepositions = dependent[3]
            for preposition in prepositions:
                for token in tokens:
                    if token.id == preposition:
                        text_token = token.text
                        lemma = morph.parse(text_token)[0]
                        prepositions[0] = lemma.normal_form
    return verbs_list

In [22]:
#Удаление id
def delete_id(verbs_list):
    for verb in verbs_list:
        verb.pop(1)
    return verbs_list

In [23]:
#Получение гиперонимов
def get_hypernyms_for_word(word):
    hypernyms = []
    synsets = wikiwordnet.get_synsets(word)
    for synset in synsets:
        synset_hypernyms = wikiwordnet.get_hypernyms(synset)
        for hypernym in synset_hypernyms:
            for word in hypernym.get_words():
                hypernym_word = word.lemma()
                hypernyms.append(hypernym_word)
    hypernyms = list(set(hypernyms))
    return hypernyms

In [24]:
#Поиск значения
def find_sense(verbs_list):
    for verb in verbs_list:
        flag = True
        search_verb = verb[0]
        dependents = verb[1]
        sub_valency_examples_table = valency_examples_table.loc[((valency_examples_table['Несовершенный вид'] == search_verb) | (valency_examples_table['Совершенный вид'] == search_verb))]
        sub_valency_semantics_table = valency_semantics_table.loc[((valency_semantics_table['Несовершенный вид'] == search_verb) | (valency_semantics_table['Совершенный вид'] == search_verb))]
        if ((sub_valency_examples_table.empty == True) & (sub_valency_semantics_table.empty == True)) :
            verb.append('Не определено')
        for dependent in dependents:
            dependent_word = dependent[0]
            dependent_POS = dependent[1]
            dependent_case = dependent[2]
            dependent_prepositions = dependent[3]
            hypernyms = get_hypernyms_for_word(dependent_word)
            for index in range (len(sub_valency_examples_table)):
                for word in sub_valency_examples_table['Примеры'].values[index]:
                    if flag == True:
                        if dependent_word == word:
                            sense = sub_valency_examples_table['Значение'].values[index]
                            verb.append(sense)
                            flag = False
            for index in range (len(sub_valency_examples_table)):
                for dependent_preposition in dependent_prepositions:
                    if flag == True:
                        if dependent_preposition == sub_valency_examples_table['Предлог'].values[index]:
                            sense = sub_valency_examples_table['Значение'].values[index]
                            verb.append(sense)
                            flag = False
            for index in range (len(sub_valency_semantics_table)):
                for word in sub_valency_semantics_table['Семантика'].values[index]:
                    if flag == True:
                        for hypernym in hypernyms:
                            if hypernym == word:
                                sense = sub_valency_examples_table['Значение'].values[index]
                                verb.append(sense)
                                flag = False
        if flag == True:
            for dependent in dependents:
                dependent_word = dependent[0]
                dependent_POS = dependent[1]
                dependent_case = dependent[2]
                dependent_preposition = dependent[3]
                for index in range (len(sub_valency_examples_table)):
                    if flag == True:
                        if dependent_case == sub_valency_examples_table['Падеж'].values[index]:
                            sense = sub_valency_examples_table['Значение'].values[index]
                            verb.append(sense)
                            flag = False
                for index in range (len(sub_valency_examples_table)):
                    if flag == True:
                        if dependent_POS == sub_valency_examples_table['Часть речи'].values[index]:
                            sense = sub_valency_examples_table['Значение'].values[index]
                            verb.append(sense)
                            flag = False
    return verbs_list

In [25]:
#Поиск английского перевода
def find_english_translation(word, sense):
    sub_english_table = english_table.loc[(((english_table['Несовершенный вид'] == word) | (english_table['Совершенный вид'] == word)) & (english_table['Значение'] == sense))]
    verb_translation = sub_english_table['Перевод'].values[0]
    return verb_translation

In [26]:
#Поиск французского перевода
def find_french_translation(word, sense):
    sub_french_table = french_table.loc[(((french_table['Несовершенный вид'] == word) | (french_table['Совершенный вид'] == word)) & (french_table['Значение'] == sense))]
    verb_translation = sub_french_table['Перевод'].values[0]
    return verb_translation

Считывание текста из текстового файла. Разделение текста на предложения:

In [36]:
file = open('Тестовая выборка 1.txt')
file = file.read()
sentences = file.split('.')

Считывание текста из excel-файла. Разделение текста на предложения:

In [ ]:
sentences = []
excel_file = open('Тестовая выборка 6.xlsx')
excel_data = pandas.read_excel('Тестовая выборка 6.xlsx')
for index in range (len(excel_data)):
    text = excel_data['Текст'].values[index]
    sentences.append(text)

Для каждого глагола в каждом предложении применяются все определённые выше функции:

In [37]:
results = pandas.DataFrame(columns = ['Предложение', 'Глагол', 'Зависимые слова', 'Значение'])
for sentence in sentences:
    doc = Doc(sentence)
    doc.segment(segmenter)
    doc.parse_syntax(syntax_parser)
    verbs_list = find_verbs(doc.tokens)
    verbs_list = find_dependents(verbs_list, doc.tokens)
    verbs_list = find_prepositions(verbs_list, doc.tokens)
    verbs_list = decipher_dependents(verbs_list, doc.tokens)
    verbs_list = decipher_prepositions(verbs_list, doc.tokens)
    verbs_list = delete_id(verbs_list)
    verbs_list = find_sense(verbs_list)
    for verb in verbs_list:
        word = verb[0]
        dependents = verb[1:(len(verb)-1)]
        sense = verb[len(verb)-1]
        line = pandas.DataFrame({'Предложение': sentence, 'Глагол': word, 'Зависимые слова': dependents, 'Значение': sense})
        results = results.append(line, ignore_index = True)

<ipython-input-37-f0c22b4d17f2>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(line, ignore_index = True)
<ipython-input-37-f0c22b4d17f2>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(line, ignore_index = True)
<ipython-input-37-f0c22b4d17f2>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(line, ignore_index = True)
<ipython-input-37-f0c22b4d17f2>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(line, ignore_index = True)
<ipython-input-37-f0c22b4d17f2>:18: FutureWarning: The frame.append method is deprecated and will be removed

Теперь каждому глаголу приписано значение ("математическое", "нематематическое" или "не определено"):

In [38]:
results

,Предложение,Глагол,Зависимые слова,Значение
0,Я возвожу число в степень,возводить,"[[число, NOUN, nomn, []]]",Математическое
1,Я возвожу двойку в куб,возводить,"[[двойка, NOUN, accs, []], [куб, NOUN, nomn, [...",Математическое
2,Мама возвела на меня напраслину,возвести,"[[мама, NOUN, nomn, []], [напраслина, NOUN, ac...",Нематематическое
3,Нужно выделить полный квадрат,выделить,"[[квадрат, NOUN, nomn, []]]",Математическое2
4,Предприятие выделило участок под огород,выделить,"[[предприятие, NOUN, nomn, []], [участок, NOUN...",Нематематическое
5,Мальчик выколол глаз терновым кустом,выколоть,"[[мальчик, NOUN, nomn, []], [глаз, NOUN, gent,...",Нематематическое
6,На этом промежутке мы выкололи точку,выколоть,"[[промежуток, NOUN, loct, [на]], [точка, NOUN,...",Математическое
7,Попробуй извлечь квадратный корень из 81,попробовать,[],Не определено
8,Попробуй извлечь квадратный корень из 81,извлечь,"[[корень, NOUN, nomn, []]]",Математическое
9,Трудно извлекать пользу из этого,извлекать,"[[польза, NOUN, accs, []]]",Нематематическое


Скачиваем результаты для оценки вручную:

In [ ]:
results.to_excel('Результаты 6.xlsx')

Для всех глаголов с математическим значением найдём верный английский перевод из таблицы переводов:

In [39]:
english_translation_results = results.loc[((results['Значение'] != 'Нематематическое') & (results['Значение'] != 'Не определено'))]
english_translation_results = english_translation_results.drop(columns = 'Зависимые слова')
english_translation_results['Перевод'] = ''
for index in range (len(english_translation_results)):
    verb = english_translation_results['Глагол'].values[index]
    sense = english_translation_results['Значение'].values[index]
    english_verb_translation = find_english_translation(verb, sense)
    english_translation_results['Перевод'].values[index] = english_verb_translation

Конечный результат для английского:

In [40]:
english_translation_results

,Предложение,Глагол,Значение,Перевод
0,Я возвожу число в степень,возводить,Математическое,raise
1,Я возвожу двойку в куб,возводить,Математическое,raise
3,Нужно выделить полный квадрат,выделить,Математическое2,complete
6,На этом промежутке мы выкололи точку,выколоть,Математическое,exclude
8,Попробуй извлечь квадратный корень из 81,извлечь,Математическое,extract
11,"Мы транспонируем матрицу, чтобы решить задачу",транспонировать,Математическое,transpose


Для всех глаголов с математическим значением найдём верный французский перевод из таблицы переводов:

In [41]:
french_translation_results = results.loc[((results['Значение'] != 'Нематематическое') & (results['Значение'] != 'Не определено'))]
french_translation_results = french_translation_results.drop(columns = 'Зависимые слова')
french_translation_results['Перевод'] = ''
for index in range (len(french_translation_results)):
    verb = french_translation_results['Глагол'].values[index]
    sense = french_translation_results['Значение'].values[index]
    french_verb_translation = find_french_translation(verb, sense)
    french_translation_results['Перевод'].values[index] = french_verb_translation

Конечный результат для французского:

In [42]:
french_translation_results

,Предложение,Глагол,Значение,Перевод
0,Я возвожу число в степень,возводить,Математическое,élever
1,Я возвожу двойку в куб,возводить,Математическое,élever
3,Нужно выделить полный квадрат,выделить,Математическое2,complèter
6,На этом промежутке мы выкололи точку,выколоть,Математическое,exclure
8,Попробуй извлечь квадратный корень из 81,извлечь,Математическое,extraire
11,"Мы транспонируем матрицу, чтобы решить задачу",транспонировать,Математическое,transposer
